# Stochastic Variational Method with RL algorithms

In [1]:
import numpy as np
import gym
import torch
import subprocess
import os
import pickle

## Expoloring environment

In [2]:
env = gym.make('svm_env:svmEnv-v2', n_pairs = 3, n_basis = 50, file_sigmas ="./svmCodeSVD/sigmas.dat" )

print('### Env Name ######', env.unwrapped.spec.id)

obs_space = env.observation_space

print('###### Observation space ####### \n', obs_space)

state_size = env.observation_space.shape[-1]

print('###### Size of observation space ####### \n', state_size)

act_space = env.action_space.shape

print('###### Action space ####### \n', act_space)

act_size = env.action_space.shape[0]*env.action_space.shape[-1]

print('###### Number of actions ####### \n', act_size)

state = env.reset()

print('##### State after reset ###### \n', state)

print('##### File where will be stored sigmas \n', env.file_sigmas)



### Env Name ###### svmEnv-v2
###### Observation space ####### 
 Box(-inf, inf, (1,), float32)
###### Size of observation space ####### 
 1
###### Action space ####### 
 (50, 3)
###### Number of actions ####### 
 150
*****CALL RESET******
Action chosen at reset:  [0.]
##### State after reset ###### 
 [0.]
##### File where will be stored sigmas 
 ./svmCodeSVD/sigmas.dat


# Your codes `DDPG` and `PPO`

## Functions for saving and clean

In [3]:
## Save all rewards, energies and princip dims in files during episode training
def create_run_fold_and_info(agent, env):
    
    # Check if folder exist and creat it
    i = 0
    while os.path.exists(f'runs_optim_envs/run_{i}/'):
        i += 1
    name_dir = f'runs_optim_envs/run_{i}/'
    os.makedirs(name_dir)
    
    # Create info.p to store info in pickle file
    info = {'alg':agent.name, 'env':env.unwrapped.spec.id , 'basis_size':env.n_basis \
            , 'batch_size':agent.batch_size, 'bootstrap_size':agent.bootstrap_size \
            , 'gamma':agent.gamma, 'tau':agent.tau,'lr_critic':agent.lr_critic \
            , 'lr_actor':agent.lr_actor, 'update_every':agent.update_every \
            , 'transfer_every':agent.transfer_every, 'num_update':agent.num_update \
            , 'add_noise_every':agent.add_noise_every}
    
    pickle.dump(info, open(name_dir+'info.p', 'wb'))
    return name_dir
    
def save_all(name_run_dir, i_ep, sigmas_i_ep, rew_i_ep, en_i_ep, pri_dim_i_ep \
             , full_dim_i_ep, act_model_i_ep, cr_model_i_ep):
    
    pickle.dump(sigmas_i_ep, open(name_run_dir+f'sigmas_{i_ep}.p', 'wb'))
    pickle.dump(rew_i_ep, open(name_run_dir+f'rew_{i_ep}.p', 'wb'))
    pickle.dump(en_i_ep, open(name_run_dir+f'en_{i_ep}.p', 'wb'))
    pickle.dump(pri_dim_i_ep, open(name_run_dir+f'pri_dim_{i_ep}.p', 'wb'))
    pickle.dump(full_dim_i_ep, open(name_run_dir+f'full_dim_{i_ep}.p', 'wb'))
    pickle.dump(act_model_i_ep, open(name_run_dir+f'act_model_{i_ep}.p', 'wb'))
    pickle.dump(cr_model_i_ep, open(name_run_dir+f'cr_model_{i_ep}.p', 'wb'))
    
def rm_useless_file(actor_model_file, critic_model_file, file_sigmas):
    os.remove(actor_model_file)
    os.remove(critic_model_file)
    os.remove(file_sigmas)

## From my `ddpg_agent.py` code

In [4]:
from ddpg_agent import DDPG_agent
agent = DDPG_agent(state_size, act_size, seed = 0)

In [5]:
## Run ddpg algs   
def run_ddpg(max_t_step = 10, n_episodes=10):
    
    # Create h5 file and store info about alg and its hypereparams
    name_run_dir = create_run_fold_and_info(agent, env)
    
    for i_ep in range(n_episodes):
        state = env.reset()
        agent.reset()
        rew_i_ep = []
        en_i_ep = []
        pri_dim_i_ep = []
        full_dim_i_ep = []
        action_i_episode = []

        ## Training loop of each episode
        for t_step in range(max_t_step):
            action = agent.act(state)
            next_state, reward, done, info = env.step(action.reshape((env.n_basis,env.n_pairs)))
            agent.step(state, action, reward, next_state, done)
            state = next_state

            # Save rew, energies, princip dims, act and crit models
            action_i_episode.append(action.reshape((env.n_basis,env.n_pairs)))
            rew_i_ep.append(reward)
            en_i_ep.append(state[0])
            pri_dim_i_ep.append(env.princp_dim)
            full_dim_i_ep.append(env.full_dim)
            torch.save(agent.actor_local.state_dict(), 'checkpoint_actor.pth')
            torch.save(agent.critic_local.state_dict(), 'checkpoint_critic.pth')
            if done:
                break
                
        ## Save data during training (to not lose the work done)
        save_all(name_run_dir=name_run_dir, i_ep=int(i_ep), sigmas_i_ep=action_i_episode \
                 , rew_i_ep=rew_i_ep, en_i_ep=en_i_ep, pri_dim_i_ep=pri_dim_i_ep \
                 , full_dim_i_ep=full_dim_i_ep, act_model_i_ep='checkpoint_actor.pth' \
                 , cr_model_i_ep='checkpoint_critic.pth')
        
        print('Episode {} ... Score: {:.3f}'.format(i_ep, np.sum(rew_i_ep)))

    rm_useless_file('checkpoint_actor.pth', 'checkpoint_critic.pth', env.file_sigmas)
    return name_run_dir

In [6]:
all_data = run_ddpg(10, 10)

*****CALL RESET******
Action chosen at reset:  [0.]
****CALL STEP****
Action chosen at step:  [[50.072845 56.929455 59.501633]
 [52.250034 34.559227 43.501587]
 [69.3533   58.621216 42.75607 ]
 [63.978046 59.135555 66.0863  ]
 [41.53401  73.24413  40.14331 ]
 [43.737583 64.73474  60.997383]
 [80.35115  49.306587 55.009674]
 [66.11502  55.713226 58.963757]
 [68.96707  71.19537  55.571434]
 [67.822685 38.9048   57.794865]
 [64.07574  53.109283 77.59395 ]
 [43.269043 50.85245  67.81748 ]
 [45.936638 47.437347 53.816032]
 [45.433517 55.54785  73.11075 ]
 [66.68302  61.524677 69.0898  ]
 [55.760532 59.582672 50.89537 ]
 [62.075413 70.3105   62.022034]
 [45.539757 47.263195 46.332912]
 [90.82975  66.02377  69.61709 ]
 [43.456844 66.45541  73.74588 ]
 [61.10525  40.024307 72.54977 ]
 [52.919365 63.2393   46.964615]
 [46.273148 59.848316 64.95668 ]
 [76.48852  70.03212  56.907513]
 [76.57767  53.520897 52.050827]
 [40.087807 70.93245  45.784874]
 [60.35365  81.0475   54.985733]
 [61.4239   42.

With this action the energy is:  -0.0714407
With this action the full dim is:  50  and princip dim is:  46
#### THE ACTION IS A GOOD ONE ####
**** THE AGENT STATE IS THE ENERGY **** -0.0714407
Set reward :  4.50741155949288
****CALL STEP****
Action chosen at step:  [[ 72.74019    61.207417   80.454956 ]
 [ 90.270004   73.301186   41.565544 ]
 [ 37.735565   60.333717   48.091087 ]
 [ 22.391567   30.91507    65.15376  ]
 [ 36.83303    43.273476   55.788685 ]
 [ 87.93541    75.86522    57.55922  ]
 [ 57.307823   63.56945    69.92623  ]
 [ 15.967724  109.310135   81.05629  ]
 [ 91.52809    30.287537   56.93226  ]
 [ 61.022594   70.27894    54.385574 ]
 [ 74.43622    58.13622    55.60167  ]
 [ 65.449554   74.58558    28.42765  ]
 [ 55.73499    30.658587   64.657425 ]
 [ 68.60836    29.391832   97.702515 ]
 [ 66.507515   55.601875   40.08684  ]
 [ 79.650375   49.15297    53.49122  ]
 [ 40.819607   54.998478   64.34801  ]
 [ 42.785694   51.633114   10.899338 ]
 [ 71.848404   31.013517   51.82

With this action the energy is:  -0.0773305
With this action the full dim is:  50  and princip dim is:  50
#### THE ACTION IS A GOOD ONE ####
**** THE AGENT STATE IS THE ENERGY **** -0.0773305
Set reward :  4.916985092216716
****CALL STEP****
Action chosen at step:  [[ 69.848495   82.43274    88.639946 ]
 [ 86.2413     77.82727    45.393547 ]
 [ 29.938496   44.43492    52.01001  ]
 [ 20.780567   67.04061    79.619354 ]
 [ 25.320467    1.         66.63853  ]
 [ 87.00914   106.14501    58.142666 ]
 [ 64.239944   70.6122     73.95979  ]
 [ 22.239788   70.07058    67.2175   ]
 [ 71.88642    73.969635   22.486969 ]
 [ 73.43607    57.412712   41.543953 ]
 [ 56.94631    42.890015   42.30543  ]
 [ 46.84287    33.264465   65.79669  ]
 [ 36.615925   19.87009    58.945473 ]
 [ 20.036938   16.218582   83.735435 ]
 [ 67.02592    48.747456   25.760368 ]
 [ 79.480484   58.355648   73.838776 ]
 [ 64.27967    24.71337    94.06642  ]
 [ 68.353424   52.793495   20.92744  ]
 [105.9727     64.79233    46.5

With this action the energy is:  -0.101608
With this action the full dim is:  50  and princip dim is:  48
#### THE ACTION IS A GOOD ONE ####
**** THE AGENT STATE IS THE ENERGY **** -0.101608
Set reward :  6.605229419817478
****CALL STEP****
Action chosen at step:  [[47.910282  65.91817   52.869972 ]
 [38.01597   70.43113   33.705574 ]
 [19.018047  40.813667  61.06211  ]
 [55.706947  92.465706  61.047997 ]
 [69.16368   64.7321    46.046814 ]
 [64.92832   52.98606   58.735188 ]
 [65.27815   54.863953  31.864119 ]
 [52.56764   51.051975  29.429878 ]
 [38.407463  44.287327  35.053123 ]
 [60.441982  78.77763   15.07066  ]
 [58.392765  52.504944  53.09008  ]
 [85.28138   57.29036   57.91792  ]
 [69.89972   28.834358  42.38278  ]
 [60.789753  36.943756  73.0849   ]
 [60.073402  60.290108  50.473232 ]
 [35.24772   81.34396   51.887585 ]
 [60.46126   57.252064  73.61586  ]
 [ 6.6646423 42.71991   39.22461  ]
 [69.434685  37.99357   79.87404  ]
 [42.944702  81.15651   32.737103 ]
 [52.812614  59

With this action the energy is:  -0.075075
With this action the full dim is:  50  and princip dim is:  48
#### THE ACTION IS A GOOD ONE ####
**** THE AGENT STATE IS THE ENERGY **** -0.075075
Set reward :  4.7601388320621085
****CALL STEP****
Action chosen at step:  [[ 15.61969   80.71357   57.212833]
 [ 35.136936  95.66864   24.288115]
 [ 27.172476  65.403915  57.731422]
 [ 75.94577   99.70474   51.63765 ]
 [ 89.056915  63.963657  44.217133]
 [ 67.55074   89.236435  86.52193 ]
 [ 67.861374  35.583794  39.418705]
 [ 54.207508  95.372604  68.11985 ]
 [ 49.899967  70.84878   55.82368 ]
 [ 79.92824   93.87468   36.978733]
 [ 84.72542   88.41527   44.80714 ]
 [ 64.16576   96.82272   85.396385]
 [ 71.81859   18.81131   35.668213]
 [ 53.02034   66.10429   48.68408 ]
 [ 60.475224 110.        62.256935]
 [ 48.85979   67.18933   90.73338 ]
 [ 45.552635  65.71806   53.967186]
 [ 44.772953  20.018223  76.69427 ]
 [ 69.29628   77.025604  58.59526 ]
 [ 64.45555   57.918575  37.5013  ]
 [ 44.873905  

...

With this action the energy is:  -0.093389
With this action the full dim is:  50  and princip dim is:  50
#### THE ACTION IS A GOOD ONE ####
**** THE AGENT STATE IS THE ENERGY **** -0.093389
Set reward :  6.033684568471554
****CALL STEP****
Action chosen at step:  [[ 39.109013   53.976864   51.72052  ]
 [ 60.463585   67.03827    45.601757 ]
 [ 33.925926   45.634804   46.147926 ]
 [ 40.983986   78.46034    73.51695  ]
 [ 30.785715    6.4982452  59.766426 ]
 [ 75.72879    51.039913   18.193214 ]
 [ 42.06144    63.27906    23.641085 ]
 [ 60.993332   18.736485   28.465555 ]
 [ 51.326385   93.58705    51.855804 ]
 [ 36.33253    61.954704   80.45743  ]
 [ 49.208954   62.060104   67.65469  ]
 [ 46.823708   55.780453   49.05209  ]
 [ 53.979813   24.813702   54.12896  ]
 [ 58.76335    71.9299     61.532104 ]
 [ 82.29196    82.44205    61.13601  ]
 [ 53.679787   25.43415    47.3871   ]
 [ 62.24784    52.68949    24.507132 ]
 [ 47.798576   22.868404   47.688744 ]
 [ 68.68362    66.81532    61.056

With this action the energy is:  -0.127659
With this action the full dim is:  50  and princip dim is:  50
#### THE ACTION IS A GOOD ONE ####
**** THE AGENT STATE IS THE ENERGY **** -0.127659
Set reward :  8.416801986116619
****CALL STEP****
Action chosen at step:  [[ 66.630974  64.313705  87.56597 ]
 [ 32.693703  58.728058  66.94182 ]
 [ 28.547619  26.815441  47.175076]
 [ 23.899454  46.926125  77.12244 ]
 [ 20.450916  32.529922  39.61525 ]
 [ 55.4041    43.975746  39.828262]
 [ 36.790466  64.02343   41.501568]
 [ 43.623173  41.28737   28.375393]
 [ 52.100292  74.03333   51.941654]
 [ 31.807579  55.981476  91.424576]
 [101.741745  25.616884  93.73393 ]
 [ 37.450085  56.339252  78.44215 ]
 [ 64.41364   21.26841   67.806946]
 [ 60.780655  91.403305  58.84205 ]
 [ 70.57737   83.62318   29.937164]
 [ 57.244003  17.67643   31.57415 ]
 [ 93.73443   54.309593  12.768738]
 [ 34.12267   39.501827  68.77689 ]
 [ 81.504105  35.91239   75.16913 ]
 [ 31.069307  63.867027  49.61257 ]
 [ 24.917725  7

With this action the energy is:  -0.076749
With this action the full dim is:  50  and princip dim is:  48
#### THE ACTION IS A GOOD ONE ####
**** THE AGENT STATE IS THE ENERGY **** -0.076749
Set reward :  4.876547894132077
****CALL STEP****
Action chosen at step:  [[35.010094  86.19047   94.933716 ]
 [51.14501   92.28937   46.418373 ]
 [50.65117   81.17784   70.91638  ]
 [75.75331   52.656036  69.93029  ]
 [83.11556   12.238152  40.29245  ]
 [73.41055   49.59772   58.110146 ]
 [51.573437  61.714584  50.42079  ]
 [19.916782  33.493313  75.67859  ]
 [67.40957   37.41964   29.220104 ]
 [25.294058  66.70994   74.10732  ]
 [50.350925  44.634087  92.17368  ]
 [66.08438   68.71976   66.69591  ]
 [74.203354  45.64144   41.0497   ]
 [81.44327   96.1172    87.3165   ]
 [86.47322   70.174255  28.319584 ]
 [47.64692   36.29316   15.2960205]
 [85.040405  62.611984  41.77728  ]
 [26.24085   78.425446  36.01673  ]
 [58.366432  73.69209   67.74269  ]
 [37.29825   64.01653   53.475803 ]
 [37.357708  57

With this action the energy is:  -0.0773378
With this action the full dim is:  50  and princip dim is:  50
#### THE ACTION IS A GOOD ONE ####
**** THE AGENT STATE IS THE ENERGY **** -0.0773378
Set reward :  4.917492730301013
****CALL STEP****
Action chosen at step:  [[ 25.574934  98.494705  90.11548 ]
 [ 42.714184  66.33938   67.634926]
 [ 26.546963  65.84837   99.54618 ]
 [ 72.38805   52.500546  51.832405]
 [ 68.492966  23.794128  13.990364]
 [ 55.5278    50.38293   77.95918 ]
 [ 54.391     50.155766  62.379986]
 [ 23.3373     9.202427  65.900444]
 [110.        40.545444  38.948326]
 [ 33.98884   84.97124   52.738598]
 [ 44.445595  34.37535  110.      ]
 [ 84.67518   69.008446  47.740486]
 [109.02443   44.491646   9.716888]
 [ 85.994385  65.37996   80.81267 ]
 [ 96.37155   73.97499   18.176441]
 [ 51.021652  17.467255   1.      ]
 [ 99.125656  50.454494  29.426916]
 [ 40.779724  54.30432   34.910423]
 [ 66.10927   52.996338  40.1663  ]
 [ 29.643993  61.827736  75.87224 ]
 [  6.250538 

With this action the energy is:  -0.136924
With this action the full dim is:  50  and princip dim is:  50
#### THE ACTION IS A GOOD ONE ####
**** THE AGENT STATE IS THE ENERGY **** -0.136924
Set reward :  9.061085116390373
Episode 9 ... Score: 65.008


## Random search as in original SVM

In [ ]:
state = env.reset()
scores = []
step = 0
score = 0.0

while True:
    print(".....STEP.....", step)
    action = env.action_space.sample()
    next_state, reward, done, info = env.step(action)
    step = step + 1
    score += reward
    scores.append(score)
    state = next_state
    if done:
        break